In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, \
average_precision_score, roc_auc_score, plot_precision_recall_curve, plot_roc_curve, plot_confusion_matrix 
from sklearn.exceptions import ConvergenceWarning

from xgboost import XGBClassifier, XGBRFClassifier, plot_importance

from imblearn.over_sampling import SMOTENC

import matplotlib.pyplot as plt

import warnings

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from IPython.display import clear_output


import pickle

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.functions import test_model, get_timeseries_table, add_model,\
add_hypersearch, return_score

from seaborn import heatmap

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [7]:
def return_score(optim_result):
    """
    callback for a hyperparameter search.  Displays current best score and 
    best parameters.  To be added to the fit method, callback argument.
    """
    score = opt.best_score_
    params = pd.DataFrame(opt.best_params_)
    clear_output()
    print('Best Score So Far: ', score)
    print('Using Parameters: ', params)

In [2]:
prediction_window = 135

df = get_timeseries_table(prediction_window=prediction_window,
                         binary_labels=True, one_hot_modules=True)

X = df.drop(columns=['final_result'])
y = df['final_result']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=111)
X_t, X_val, y_t, y_val = train_test_split(X_train, y_train, random_state=111)

all_features = 'SMOTED activites, clicks, activities*clicks, assessments, modules'

categoricals = [502, 501, 500, 499, 498, 497, 496]
smotenc = SMOTENC(categoricals, random_state=111)
X_train, y_train = smotenc.fit_resample(X_train, y_train)
X_t, y_t = smotenc.fit_resample(X_t, y_t)

X_t_normal = normalize(X_t)
X_val_normal = normalize(X_val)
X_train_normal = normalize(X_train)
X_test_normal = normalize(X_test)

assessments merged:  1


In [4]:
models = [LogisticRegression(random_state=111, solver='liblinear'),
         DecisionTreeClassifier(random_state=111),
         RandomForestClassifier(random_state=111),
         KNeighborsClassifier(),
         SVC(random_state=111, probability=True),
         SGDClassifier(loss='log', random_state=111),
         AdaBoostClassifier(random_state=111),
         XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=111),      
         ]

for model in models:
    model.fit(X_t, y_t)
    add_model(model, X_t, y_t, X_val, y_val, 
              preprocessing=None, 
              features = all_features)


In [5]:
models = [LogisticRegression(random_state=111, solver='liblinear'),
         DecisionTreeClassifier(random_state=111),
         RandomForestClassifier(random_state=111),
         CategoricalNB(),
         GaussianNB(),
         KNeighborsClassifier(),
         SVC(random_state=111, probability=True),
         AdaBoostClassifier(random_state=111),
         XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=111)      
         ]

for model in models:
    model.fit(X_t_normal, y_t)
    add_model(model, X_t_normal, y_t, X_val_normal, y_val, 
              features = all_features,
              preprocessing='normalized')


h_table = None

In [31]:
xgb = XGBClassifier(use_label_encoder=False, seed=111, eval_metric='logloss')

xgb_search_space = {'objective': Categorical(['binary:logistic',
                                              'binary:hinge']),
                    'eval_metric': Categorical(['logloss','error']),
                   'learning_rate': Real(0.05, .3, 'log-uniform'),
                   'min_child_weight': Integer(1,10, 'uniform'),
                   'max_depth': Integer(20,60, 'normal'),
                   'subsample': Real(0.3, 1, 'normal'),
                   'colsample_bytree': Real(.3, 1.0, 'normal'),
                   'scale_pos_weight': Real(.5, 2.0, 'uniform')}

opt = BayesSearchCV(xgb, search_spaces=xgb_search_space, 
                    n_iter=50, cv=2,
                    n_jobs=4,
                    pre_dispatch = 8,
                    random_state=111,
                    return_train_score=True,
                    scoring='roc_auc')

opt.fit(X_t, y_t, callback=[return_score])

XGBmodel = opt.best_estimator_

add_hypersearch(opt)

add_model(XGBmodel, X_t, y_t, X_val, y_val, 
          features = all_features,
          preprocessing=None)

Best Score So Far:  0.9344487694759113
Using Parameters:  OrderedDict([('colsample_bytree', 0.6011694082627778), ('eval_metric', 'logloss'), ('learning_rate', 0.10937423148527929), ('max_depth', 20), ('min_child_weight', 1), ('objective', 'binary:logistic'), ('scale_pos_weight', 2.0), ('subsample', 0.8574053108878759)])
[0.02159542 0.00311609 0.22038937 ... 0.08963265 0.00375508 0.01662309]


,model,val_roc_auc,train_roc_auc,val_accuracy,train_accuracy,val_f1_score,train_f1_score,features,preprocessing,C,...,subsample,tree_method,validate_parameters,verbosity,eval_metric,class_prior,fit_prior,priors,var_smoothing,seed
0,XGBClassifier,0.934449,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.857405,NaN,NaN,NaN,logloss,NaN,NaN,NaN,NaN,NaN
1,XGBClassifier,0.934330,0.999999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,logloss,NaN,NaN,NaN,NaN,NaN
2,XGBClassifier,0.933969,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,logloss,NaN,NaN,NaN,NaN,NaN
3,XGBClassifier,0.933770,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,error,NaN,NaN,NaN,NaN,NaN
4,XGBClassifier,0.933645,0.999990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,logloss,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,DecisionTreeClassifier,0.716129,1.000000,0.724506,1.000000,0.644087,0.644087,"SMOTED activites, clicks, activities*clicks, a...",normalized,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,SGDClassifier,0.680692,0.697458,0.701622,0.695484,0.594137,0.594137,"SMOTED activites, clicks, activities*clicks, a...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,KNeighborsClassifier,0.674254,0.906501,0.561875,0.737933,0.575914,0.575914,"SMOTED activites, clicks, activities*clicks, a...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,KNeighborsClassifier,0.652348,0.901600,0.556543,0.750835,0.561319,0.561319,"SMOTED activites, clicks, activities*clicks, a...",normalized,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
def add_model(model, X_t, y_t, X_val, y_val,
              preprocessing=None, 
              features='Not Provided'):
    """
    Scores a model by several metrics and saves it to a hyperparameter table
    """
    train_probs = model.predict_proba(X_t)[:,1]
    train_yhat = np.round_(train_probs)
    val_probs = model.predict_proba(X_val)[:,1]
    val_yhat = np.round_(val_probs)
    print(val_probs)
    parameters = {'model' : type(model).__name__,
                  'val_roc_auc' : roc_auc_score(y_val, val_probs), 
                  'train_roc_auc': roc_auc_score(y_t, train_probs), 
                  'val_accuracy': accuracy_score(y_val, val_yhat),
                  'train_accuracy': accuracy_score(y_t, train_yhat),
                  'val_f1_score': f1_score(y_val, val_yhat),
                  'train_f1_score': f1_score(y_val, val_yhat),
                  'features': features,
                  'preprocessing': preprocessing,
                 }
    parameters.update(model.get_params())
    parameters = pd.DataFrame(parameters, index=[0])

    try:
        table = pd.read_csv('hyperparameter_table.csv')
    except:
        table = pd.DataFrame()
    table = table.append(parameters, ignore_index=True)
    table = table.drop_duplicates(subset=table.columns[7:], keep='last')
    table = table.sort_values(by='val_roc_auc', ascending=False)
#     table.to_csv('hyperparameter_table.csv', index=False)
    display(table)

In [33]:
opt2 = opt.best_params_

In [34]:
add_model(XGBmodel, X_t, y_t, X_val, y_val, 
          features = all_features,
          preprocessing=None)

[0.02159542 0.00311609 0.22038937 ... 0.08963265 0.00375508 0.01662309]


,model,val_roc_auc,train_roc_auc,val_accuracy,train_accuracy,val_f1_score,train_f1_score,features,preprocessing,C,...,subsample,tree_method,validate_parameters,verbosity,eval_metric,class_prior,fit_prior,priors,var_smoothing,seed
0,XGBClassifier,0.934449,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.857405,NaN,NaN,NaN,logloss,NaN,NaN,NaN,NaN,NaN
1,XGBClassifier,0.934330,0.999999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,logloss,NaN,NaN,NaN,NaN,NaN
2,XGBClassifier,0.933969,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,logloss,NaN,NaN,NaN,NaN,NaN
3,XGBClassifier,0.933770,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,error,NaN,NaN,NaN,NaN,NaN
4,XGBClassifier,0.933645,0.999990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,NaN,NaN,NaN,logloss,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,DecisionTreeClassifier,0.716129,1.000000,0.724506,1.000000,0.644087,0.644087,"SMOTED activites, clicks, activities*clicks, a...",normalized,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,SGDClassifier,0.680692,0.697458,0.701622,0.695484,0.594137,0.594137,"SMOTED activites, clicks, activities*clicks, a...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,KNeighborsClassifier,0.674254,0.906501,0.561875,0.737933,0.575914,0.575914,"SMOTED activites, clicks, activities*clicks, a...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,KNeighborsClassifier,0.652348,0.901600,0.556543,0.750835,0.561319,0.561319,"SMOTED activites, clicks, activities*clicks, a...",normalized,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rf = RandomForestClassifier(random_state=111)

rf_search_space = {'n_estimators': Integer(50, 500, 'normal'),
                   'max_depth': Integer(5, 50, 'normal'),
                   'max_features': Categorical(['sqrt','log2',None]),
                   'class_weight': Categorical(['balanced','balanced_subsample',None]),
                   'max_samples': Real(.1, .99, 'uniform')
                   }

opt = BayesSearchCV(rf, search_spaces=rf_search_space, 
                    n_iter=50, cv=2,
                    n_jobs=4,
                    pre_dispatch = 8,
                    random_state=111,
                    return_train_score=True,
                    scoring='roc_auc')

opt.fit(X_t, y_t, callback=[return_score])

RFclf = opt.best_estimator_

add_hypersearch(opt)

add_model(RFclf, X_t, y_t, X_val, y_val, 
          features=all_features, 
          preprocessing=None)

Best Score So Far:  0.9286259802313596
Using Parameters:  OrderedDict([('class_weight', None), ('max_depth', 46), ('max_features', 'sqrt'), ('max_samples', 0.99), ('n_estimators', 500)])


In [ ]:
DT = DecisionTreeClassifier(random_state=111)

DT_search_space = {'criterion': Categorical(['gini','entropy']),
                   'splitter': Categorical(['best','random']),
                    'max_depth': Integer(3,10, 'uniform'),
                   'max_features': Categorical(['sqrt','log2', None]),
                   'class_weight': Categorical(['balanced',None]),
                   }

opt = BayesSearchCV(DT, search_spaces=DT_search_space, 
                    n_iter=50, cv=2,
                    n_jobs=4,
                    pre_dispatch = 8,
                    random_state=111,
                    scoring='roc_auc',
                    return_train_score=True)

opt.fit(X_t, y_t, callback=[return_score])

DTclf = opt.best_estimator_

add_hypersearch(opt)

add_model(DTclf, X_t, y_t, X_val, y_val, 
          features=all_features, 
          preprocessing=None)

In [ ]:
table[table['model'] == 'XGBClassifier'].dropna(subset=['val_accuracy'], how='any')

In [ ]:
table = pd.read_csv('hyperparameter_table.csv')
table.head(20)

In [ ]:
clf=XGBmodel

In [ ]:
y_score = clf.predict_proba(X_val)[:,1]

fig, axes = plt.subplots(1,2, figsize=(10,5))
plot_precision_recall_curve(clf, X_val, y_val, ax = axes[0])
axes[0].set_title('Precision-Recall Curve')

plot_roc_curve(clf, X_val, y_val, ax= axes[1])
axes[1].set_title('ROC Curve')
plt.show()

In [ ]:
thresh_accuracy = pd.DataFrame(columns= ['accuracy', 'f1_score'],
                              index = [x/100 for x in range(60,80)])

thresh_accuracy.index.name = 'Threshold'
for thresh in range(10, 90):
    thresh /= 100
    yhat = pd.Series(y_score).apply(lambda x: 1 if x >= thresh else 0)
    accuracy = accuracy_score(y_val, yhat)
    f1 = f1_score(y_val, yhat)
    roc = roc_auc_score(y_val, y_score)
    pr = average_precision_score(y_val, y_score)
    thresh_accuracy.loc[thresh, 'accuracy'] = accuracy
    thresh_accuracy.loc[thresh, 'f1_score'] = f1

print('ROC_AUC: ', roc)
print('PR_AUC: ', pr)
thresh_accuracy = thresh_accuracy.sort_values(by='f1_score', ascending=False)
thresh_accuracy.head(20)

In [ ]:
yhat = pd.Series(y_score)
best_thresh = thresh_accuracy.index[0]
yhat = yhat.apply(lambda x: 1 if x >= best_thresh else 0)

confusion  = confusion_matrix(y_val, yhat, normalize='true')
heatmap(confusion, annot=True, cmap='Greens')

In [ ]:
importance = pd.DataFrame(XGBmodel.feature_importances_, index=X_t.columns)
importance.sort_values(by=0, ascending=False).head(20)

import pickle
pickle.dump(clf, open('time_series_xgb_best.pkl','wb'))